In [ ]:
import pymc as pm
import arviz as az
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
import numpy as np
import xarray as xr
from ridgeplot import ridgeplot
import plotly.express as px
import warnings

In [ ]:
RANDOM_SEED = 8927
rng = np.random.default_rng(RANDOM_SEED)
az.style.use("arviz-darkgrid")
warnings.simplefilter(action="ignore", category=FutureWarning)

In [ ]:
df = pd.read_csv("./data/writing_center_v2.csv")
df.rename(columns={'Ethnicity_Hispanic / Latino':'Ethnicity_Hispanic', 'Ethnicity_Mixed Ethnicity':'Ethnicity_Mixed','Ethnicity_White, Non-Hispanic':'Ethnicity_White'},inplace=True)
df['Main_Course_SuccessFlag'] = df['Main_Course_SuccessFlag'].astype(int).values
df['Main_Course_GradePoints'] = df['Main_Course_GradePoints'].astype(int).values

In [ ]:
def plot_parameters(trace, trace_plot=True, forest_plot=True):
  '''
  Can only be used for multilevel.
  '''
  var_names = list(trace.posterior.data_vars.keys())[:-1]
  if trace_plot:
    az.plot_trace(trace, var_names=var_names)
  if forest_plot:
    az.plot_forest(trace, var_names=var_names, combined=True)

# Complete Pooling Ordinal Logistic Regression

In [ ]:
def complete_pooling_ordinal(features, df, save_name:str, rng, cv=False):
  coords = {"obs_id": df.index.values}

  with pm.Model(coords=coords) as ordinal_regression_model:
      # Define the design matrix (X)
      X = pm.Data("X", df[features].values,
                  dims=["obs_id", "predictor"])

      # Priors
      cutpoints = pm.Normal("cutpoints", mu=[0, 0.5, 1, 1.5], sigma=1, shape=4,
                          transform=pm.distributions.transforms.ordered)
      beta_distributions = {} 
      # Create Normal distributions dynamically
      for feature in features:
          beta_name = 'beta_' + feature
          beta_distributions[beta_name] = pm.Normal(str(beta_name), mu=0, sigma=1)

      mu = 0
      # Linear model with beta0 + linear combination of features-coefficients and feature-data
      for feature in features:
        mu += beta_distributions['beta_' + feature] * X[:, features.index(feature)]

      # Likelihood
      y_obs = pm.OrderedLogistic("y_obs", eta=mu, cutpoints=cutpoints, observed=df['Main_Course_GradePoints'],dims="obs_id")



  # Inference
  with ordinal_regression_model:
      trace = pm.sample(4000, tune=2000,return_inferencedata=True, random_seed=rng)
      pm.compute_log_likelihood(trace) # used for model comparison
  
      if cv == False:  
        #save trace
        trace.to_netcdf("./traces/"+save_name+"_ordinal_complete_pooling.nc")

  return trace, ordinal_regression_model

## No Pooling Ordinal Logistic Regression

In [ ]:
def no_pooling_ordinal(features, df, save_name:str, rng, cv=False):
    instructor_idxs, instructors = pd.factorize(df.Instructor_ID)
    num_instructors = len(instructors)

    coords = {
        "instructor": instructors,
        "obs_id": np.arange(len(instructor_idxs)),
        "predictor": features
    }

    with pm.Model(coords=coords) as ordinal_regression_model:
        # Define instructor
        instructor_idx = pm.Data("instructor_idx", instructor_idxs, dims="obs_id")

        # Define the design matrix (X) with instructor-specific predictors
        X = pm.Data("X", df[features].values, dims=["obs_id", "predictor"])

        # Priors
        # beta0 = pm.Normal("beta0", 0, sigma=1, dims="instructor")  # intercept varying by instructor
        cutpoints = pm.Normal("thresholds", mu=[0, 0.5, 1, 1.5], sigma=1, shape=(num_instructors,4),
                          transform=pm.distributions.transforms.ordered,dims=("instructor","cutpoints"))

        beta_distributions = {}
        # Create Normal distributions dynamically
        for feature in features:
            beta_name = 'beta_' + feature
            beta_distributions[beta_name] = pm.Normal(str(beta_name), 0, sigma=1, dims="instructor")  # all betas varying by instructor

        # Linear model
        mu = 0 
        for feature in features:
            mu += beta_distributions['beta_' + feature][instructor_idx] * X[:, features.index(feature)]

        # Likelihood
        y_obs = pm.OrderedLogistic("y_obs", eta=mu, cutpoints=cutpoints[instructor_idx, :], observed=df['Main_Course_GradePoints'], dims="obs_id")

    # Inference
    with ordinal_regression_model:
        trace = pm.sample(4000, tune=1000, return_inferencedata=True, random_seed=rng)
        pm.compute_log_likelihood(trace)  # used for model comparison
    
        if cv == False:
            #save trace
            trace.to_netcdf("./traces/"+save_name+"_ordinal_no_pooling.nc")
    return trace, ordinal_regression_model

## Multilevel Ordinal Logistic Regression (Hierarchical)

In [ ]:
def multilevel_ordinal(features, df, save_name:str, rng, cv=False):
    instructor_idxs, instructors = pd.factorize(df.Instructor_ID)
    num_instructors = len(instructors)

    coords = {
        "instructor": instructors,
        "obs_id": np.arange(len(instructor_idxs)),
        "predictor": features
    }

    with pm.Model(coords=coords) as ordinal_regression_model:
        # Define instructor
        instructor_idx = pm.Data("instructor_idx", instructor_idxs, dims="obs_id")

        # Define the design matrix (X) with instructor-specific predictors
        X = pm.Data("X", df[features].values, dims=["obs_id", "predictor"])

        # Hyperpriors for cutpoints
        mu_cutpoints = pm.Normal("mu_cutpoints", mu=[0, 0.5, 1, 1.5], sigma=1, shape=4)
        sigma_cutpoints = pm.HalfNormal("sigma_cutpoints", sigma=1, shape=4)
        
        # Priors for instructor-specific cutpoints
        cutpoints_offset = pm.Normal("cutpoints_offset", mu=[0, 0.5, 1, 1.5], sigma=1, 
                                     dims=("instructor", "cutpoints"), 
                                     shape=(num_instructors, 4))
        cutpoints = pm.Deterministic("thresholds", mu_cutpoints + cutpoints_offset * sigma_cutpoints, 
                                     dims=("instructor", "cutpoints"))


        # Hyperpriors for intructor betas
        hyperpriors_mu = {}
        hyperpriors_sigma =  {}
        for feature in features:
            beta_name = 'beta_' + feature
            hyperpriors_mu[beta_name] = pm.Normal(str(beta_name)+'_mu', mu=0, sigma=1)
            hyperpriors_sigma[beta_name] = pm.HalfNormal(str(beta_name)+'_sigma', 1)

        # Priors  
        # mu and sigma follows from multilevel and intercept varying by instructor
        beta_distributions = {}
        # Create Normal distributions dynamically
        for feature in features:
            beta_name = 'beta_' + feature
            beta_j_offset = pm.Normal(str(beta_name)+'_offset', mu=0, sigma=1, dims="instructor")
            beta_distributions[beta_name] = pm.Deterministic(str(beta_name), hyperpriors_mu[beta_name] + beta_j_offset * hyperpriors_sigma[beta_name], dims="instructor")

        # Linear model
        mu = 0
        for feature in features:
            mu += beta_distributions['beta_' + feature][instructor_idx] * X[:, features.index(feature)]

        # Likelihood
        y_obs = pm.OrderedLogistic("y_obs", eta=mu, cutpoints=cutpoints[instructor_idx, :], 
                                   observed=df['Main_Course_GradePoints'], dims="obs_id")

    # Inference
    with ordinal_regression_model:
        print(ordinal_regression_model.debug())
        trace = pm.sample(4000, tune=1000, return_inferencedata=True, random_seed=rng, target_accept=0.99)
        pm.compute_log_likelihood(trace)  # used for model comparison
    
        if cv == False:
            #save trace
            trace.to_netcdf("./traces/"+save_name+"_ordinal_multilevel.nc")

    return trace, ordinal_regression_model

##### Run Genders

In [ ]:
gender_features = ['Gender_Male', 'Gender_Female']
# gender_complete_pooling_ordinal_trace, gender_complete_pooling_ordinal_model = complete_pooling_ordinal(gender_features, df, 'gender',rng)
# gender_no_pooling_ordinal_trace, gender_no_pooling_ordinal_model = no_pooling_ordinal(gender_features, df, 'gender',rng)
gender_multilevel_ordinal_trace, gender_multilevel_ordinal_model = multilevel_ordinal(gender_features, df, 'gender',rng)

In [ ]:
gender_multilevel_ordinal_model.debug()

In [ ]:
# Numerical Model Summary
gender_cp_ordinal_summary = pm.summary(gender_no_pooling_ordinal_trace)

In [ ]:
gender_cp_ordinal_summary

In [ ]:
plot_parameters(gender_no_pooling_ordinal_trace)

##### Run Ethnicity

In [ ]:
# Features
ethnicity_features = ['Ethnicity_White', 'Ethnicity_Asian', 'Ethnicity_Hispanic', 'Ethnicity_Mixed']
# ethnicity_complete_pooling_ordinal_trace, ethnicity_complete_pooling_ordinal_model = complete_pooling_ordinal(ethnicity_features, df, 'gender',rng)
# ethnicity_no_pooling_ordinal_trace, ethnicity_no_pooling_ordinal_model = no_pooling_ordinal(ethnicity_features, df, 'gender',rng)
# ethnicity_multilevel_ordinal_trace, ethnicity_multilevel_ordinal_model = multilevel_ordinal(ethnicity_features, df, 'gender',rng)

In [ ]:
# Numerical Model Summary
ethnicity_cp_ordinal_summary = pm.summary(ethnicity_complete_pooling_ordinal_trace)


In [ ]:
plot_parameters(ethnicity_complete_pooling_ordinal_trace)

##### Run Adjusted

In [ ]:
adjusted_features = ['Gender_Male', 'Gender_Female', 'Ethnicity_White', 'Ethnicity_Asian', 'Ethnicity_Hispanic', 'Ethnicity_Mixed','FinAid','Age','TermUnitsAttempted']
                    #   'Age', 'FirstGen', 'Military', 'FosterYouth', 'DSPS','FinAid', 'Units_Attempted_Beg_Of_Term', 
                    #   'TermUnitsAttempted', 'K12_Student', 'First_Time_College_Student', 'Nonresident_Tuition_Exempt', 'International', 'Nonresident', 
                    #   'WR_Center', 'Online', 'N_Center_Visits', 'Center_Attendance_Hours', 'N_Conf', 'WR_Center_FailFlag']
# adjusted_complete_pooling_ordinal_trace, adjusted_complete_pooling_ordinal_model = complete_pooling_ordinal(adjusted_features, df, 'adjusted', rng)
# adjusted_no_pooling_ordinal_trace, adjusted_no_pooling_ordinal_model = no_pooling_ordinal(adjusted_features, df.iloc[0:50], 'adjusted', rng)
adjusted_multilevel_ordinal_trace, adjusted_multilevel_ordinal_model = multilevel_ordinal(adjusted_features, df, 'adjusted', rng)

In [ ]:
# Numerical Model Summary
adjusted_cp_ordinal_summary = pm.summary(adjusted_multilevel_ordinal_trace)


In [ ]:
plot_parameters(adjusted_multilevel_ordinal_trace, adjusted_features)